## Project Objective

- Predict the most suitable berth for an incoming vessel before arrival using historical vessel and operational data.
- The goal is to support port operators in improving berth allocation efficiency,reducing waiting time, and optimizing terminal productivity.


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")


# Load Dataset

In [ ]:
df = pd.read_csv("/kaggle/input/vesseldata/VESSELDATA.csv")

df.head()


In [ ]:
df.shape
df.info()


### Initial Observations

- The dataset contains 7,022 records and 66 features.
- The target variable is **BerthID**, representing the assigned berth for each vessel call.
- The dataset includes a mix of numerical, categorical, temporal, and vessel-specific features.
- Several columns contain missing values, especially operational and time-related features.
- Some vessel dimension features are stored as object types and will require conversion to numerical format during preprocessing.

# Target Variable Analysis (BerthID)

In [ ]:
df['BerthID'].nunique()

In [ ]:
df['BerthID'].value_counts()


In [ ]:
plt.figure(figsize=(10,4))
df['BerthID'].value_counts().plot(kind='bar')
plt.title("Distribution of BerthID")
plt.ylabel("Number of Vessel Calls")
plt.show()


### Target Variable Distribution

The berth allocation problem involves 11 different berth classes.
The distribution shows a moderate class imbalance, where berths A and B
handle the majority of vessel calls, while other berths are used less frequently.

This reflects real-world port operations, where certain berths are preferred
due to infrastructure, location, or vessel compatibility.


# Missing Values Analysis

In [ ]:
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Percentage': missing_pct
}).sort_values(by='Missing Percentage', ascending=False)

missing_df[missing_df['Missing Count'] > 0]


### Missing Values Analysis

The dataset contains several features with varying levels of missing values.
Columns with more than 80% missing data (e.g., Anchorage Time features) were identified
as non-essential for berth allocation and will be excluded from modeling.

Some operational and post-berthing performance features show moderate missing rates
(20%–40%). While informative, these variables may introduce data leakage since they
are not available prior to vessel arrival.

Productivity and container-related features (TE variables) have approximately 19%
missing values and are considered critical for berth allocation decisions.
These features will be retained and handled using appropriate imputation strategies.

This selective approach ensures data quality while preserving meaningful operational signals.


# Feature Grouping

In [ ]:
identifier_features = [
    'Call_Key', 'Call_ID', 'VesselName', 'IMO',
    'RadioCode', 'ShipOwner_Id', 'ShipOwnerName'
]

temporal_features = [
    col for col in df.columns if 'Dt' in col or 'Duration' in col
]

vessel_physical_features = [
    'Ship type', 'VesselClass', 'LOA',
    'Length Overall (m)', 'Beam (m)',
    'Gross Tonnage', 'Summer Deadweight (t)', 'Berth_len'
]

container_features = [
    'TE_TC20', 'TE_TC40', 'TE_DEBA', 'TE_EMBA',
    'TE_DEBA_OOG', 'TE_EMBA_OOG', 'TE_GearBox',
    'TE_Hatch', 'TE_SBB', 'TE_SBTB', 'TE_Twin_Lift'
]

productivity_features = [
    'Vessel_Gross_Prdvty', 'Vessel_Net_Prdvty',
    'Berth_Prdvty', 'Lean_Operation_Time',
    'Net_Operation_Time', 'Operation_Time',
    'IdleTimeBeforeOps', 'IdleTimeAfterOps'
]

service_features = [
    'Service_ID', 'Service_Nm', 'CountryCode',
    'Type', 'IsSpecificCall', 'ShipOwnerGroup',
    'Terminal_Site_Key', 'BerthCustomGrouping'
]


# Feature Grouping Strategy

- Features were grouped based on their business meaning and availability at prediction time.
- Identifier and temporal features were excluded to prevent data leakage.
- Vessel physical characteristics and container-related variables were retained as core predictors.
- Operational and service-related features were selectively included based on their relevance to pre-arrival berth allocation decisions.


# Feature Availability & Data Leakage

- Since the objective is to predict the most suitable berth before vessel arrival, only features available at prediction time were considered.

- Post-arrival operational and time-dependent variables, such as operation times, productivity metrics, and vessel movement timestamps, were identified as potential sources of data leakage and were therefore excluded from the modeling process.


# Data Cleaning

### Goals
- Prevent data leakage by removing features not available before vessel arrival.
- Prepare the dataset for machine learning modeling.
- Retain only features available at prediction time.
- Reduce noise and inconsistencies while preserving important business information.

In [ ]:
leakage_cols = [
    'Call_Key', 'Call_ID', 'IMO', 'RadioCode', 'VesselName',
    'Arvl_Dt', 'Arvl_Dt_Year', 'Arvl_Dt_YearMonth', 'Arvl_Dt_YearMonthDay',
    'Dptr_Dt', 'Dptr_Dt_Year', 'Dptr_Dt_YearMonth', 'Dptr_Dt_YearMonthDay',
    'First_Movement_Dt', 'First_Movement_Dt_Year',
    'First_Movement_Dt_YearMonth', 'First_Movement_Dt_YearMonthDay',
    'Last_Movement_Dt', 'Last_Movement_Dt_Year',
    'Last_Movement_Dt_YearMonth', 'Last_Movement_Dt_YearMonthDay',
    'Last_Movement_Dt_WeekThursday',
    'Operation_Time', 'Net_Operation_Time', 'Lean_Operation_Time',
    'Berth_Operation_Time', 'Berth_Net_Operation_Time',
    'Vessel_Gross_Prdvty', 'Vessel_Net_Prdvty', 'Berth_Prdvty',
    'IdleTimeBeforeOps', 'IdleTimeAfterOps',
    'AnchorageTimeInHours', 'AnchorageTimeInDay'
]

df_clean = df.drop(columns=leakage_cols, errors='ignore')
df_clean.shape



After removing data leakage and post-arrival features, the dataset was reduced from 66 to 32 features.
The remaining features are available before vessel arrival and suitable for real-world berth allocation prediction.


In [ ]:
df_clean.columns

In [ ]:
df_clean.info()

# Fix Data Types

In [ ]:
numeric_cols_to_fix = [
    'Gross Tonnage',
    'Summer Deadweight (t)',
    'Length Overall (m)',
    'Beam (m)'
]

for col in numeric_cols_to_fix:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')


In [ ]:
df_clean[numeric_cols_to_fix].info()

### Data Type Correction
Several vessel physical characteristics were originally stored as object types.
These features were successfully converted to numerical format to ensure proper
statistical analysis and model compatibility.


# Missing Values

In [ ]:
missing_ratio = df_clean.isna().mean().sort_values(ascending=False)
missing_ratio


## Missing Values Handling 
- Missing values were handled based on business logic and feature characteristics.
- Container-related features were filled with zero to indicate absence of containers,while categorical features were filled with 'Unknown'.
- Numerical features with low missing ratios were imputed using the median to preserve distribution stability.


In [ ]:
te_cols = [col for col in df_clean.columns if col.startswith("TE_")]
df_clean[te_cols] = df_clean[te_cols].fillna(0)

In [ ]:
cat_cols = df_clean.select_dtypes(include="object").columns
df_clean[cat_cols] = df_clean[cat_cols].fillna("Unknown")


In [ ]:
num_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
num_cols = num_cols.drop("Berth_len", errors="ignore")  

for col in num_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())


In [ ]:
df_clean.isna().sum().sum()

# Categorical Cardinality Analysis

In [ ]:
cat_cols = df_clean.select_dtypes(include="object").columns
df_clean[cat_cols].nunique().sort_values(ascending=False)


In [ ]:
df_clean = df_clean.drop(columns=["ShipOwnerName", "ShipOwner_Id"])

In [ ]:
def group_rare_categories(df, col, min_freq=0.01):
    freq = df[col].value_counts(normalize=True)
    rare = freq[freq < min_freq].index
    df[col] = df[col].replace(rare, "Other")
    return df

for col in ["VesselClass", "Service_Nm", "Service_ID"]:
    df_clean = group_rare_categories(df_clean, col)


# Categorical Cardinality Reduction

- High-cardinality categorical features were analyzed to identify rarely occurring categories.
- Categories with very low frequency were grouped into a single "Other" class.
- This step helps reduce feature sparsity, control model complexity, and improve generalization.


In [ ]:
cat_cols = df_clean.select_dtypes(include="object").columns

for col in cat_cols:
    df_clean[col] = df_clean[col].str.strip().str.lower()


# Categorical Value Standardization

- Categorical values were standardized by removing extra spaces and converting text to lowercase.
- This ensures consistency across category labels and prevents duplicated representations of the same category.


# Final Dataset Check

In [ ]:
df_clean.shape

In [ ]:
df_clean.isna().sum().sum()

In [ ]:
df_clean[cat_cols].nunique().sort_values(ascending=False)

## Final Dataset Summary

- The dataset was cleaned and prepared for modeling with no missing values remaining.
- Data leakage was prevented by excluding post-arrival operational features.
- Categorical variables were standardized and high-cardinality features were reduced.
- The final dataset is ready for encoding and machine learning modeling.


In [ ]:
# Save cleaned dataset for modeling
df_clean.to_csv("df_clean.csv", index=False)


- After completing data cleaning, preprocessing, and categorical cardinality reduction, the final cleaned dataset was saved for downstream modeling tasks.

- This cleaned dataset will be used as the single source of truth in the modeling and evaluation notebook.